In [3]:
import pandas as pd

In [15]:
df = pd.read_csv('test2.csv', index_col="player_id").reset_index().drop_duplicates()
#df = df.pivot_table(values="value", columns="game_title", index="player_id").fillna(0)
#df = df.apply(lambda x: x != 0, axis=0).melt(ignore_index=False).reset_index()
#df["value"] = df["value"].replace({False: 0, True: 1})
# df = df.melt(ignore_index=False).reset_index()
# df = df[df["value"] != 0]
df.pivot_table(values="value", columns="game_title", index="player_id")
#df["value"] = pd.qcut(x=df["value"],q=5, labels=[1,2,3,4,5])
#print(df["value"].value_counts())
#df

game_title,15 Days,4 Elements,7 Days to Die,APB Reloaded,Act of Aggression,Age of Empires Online,Airline Tycoon 2,Alan Wake,Alone in the Dark,Alpha Protocol,...,World in Conflict,World of Goo,World of Zoo,Worldwide Soccer Manager 2009,Worms,Worms Armageddon,Worms Reloaded,X Rebirth,X-Blades,Zombie Army Trilogy
player_id,,,,,,,,,,,,,,,,,,,,,
5250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.4,NaN,NaN,NaN,NaN
86540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307181486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308118796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308220794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [13]:
from surprise import accuracy, Dataset, SVDpp, Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["player_id", "game_title", "value"]], reader)

trainset, testset = train_test_split(data, test_size=0.25)

# We'll use the famous SVD algorithm.
algo = SVDpp()

#res = cross_validate(algo, data, measures=["RMSE", "MAE"], cv=5, verbose=True)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
print(accuracy.rmse(predictions))
#print(res)

RMSE: 1.2648
1.2648126238030015


In [11]:
genres = df["game_title"].unique()
player_id = 2753525
res = {}
for genre in genres:
    res[genre] = [algo.predict(player_id, genre).est]
    # print(algo.predict(player_id, genre))
res = pd.DataFrame(data = res).T
print(res.sort_values(by=0, ascending=False).head(20))
print(df[(df["player_id"] == player_id) & (df["game_title"] == "Borderlands 2")].sort_values(by="value", ascending=False))

                                  0
Football Manager 2012      4.423048
Football Manager 2013      4.139353
Grand Theft Auto V         3.968737
Football Manager 2014      3.955688
Football Manager 2015      3.955178
Football Manager 2011      3.900609
Fallout 4                  3.765321
Rocket League              3.694125
Football Manager 2010      3.617040
Pro Evolution Soccer 2015  3.577595
EVE Online                 3.568473
NBA 2K15                   3.329233
Rocksmith 2014             3.321005
Farming Simulator 2013     3.306834
Fable III                  3.303638
Far Cry 4                  3.292317
Terraria                   3.283741
NBA 2K16                   3.275160
NBA 2K14                   3.268616
The Crew                   3.253439
Empty DataFrame
Columns: [player_id, game_title, value]
Index: []
